In [ ]:
from elasticsearch import Elasticsearch, RequestsHttpConnection, helpers
from requests_aws4auth import AWS4Auth
import boto3
import csv

In [ ]:
datapath = 'crash_db.csv'
host = 'search-govhack2018-u3oyqcrcukqglza4uqijk7xl5m.ap-southeast-2.es.amazonaws.com'
region = 'ap-southeast-2'
service = 'es'
access_key = ''
secret_key = ''

In [ ]:
# Example of One Entry
# pip install elasticsearch==6.3.1
#credentials = boto3.Session().get_credentials()

awsauth = AWS4Auth(access_key, secret_key, region, service)

es = Elasticsearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)


In [ ]:
es.indices.delete(index = 'crashes')

In [ ]:
mapping = '''
{
"mappings": {
  "crash": {
    "properties": {
      "Long": {
        "type": "text"
      },
      "Lat": {
        "type": "text"
      },
      "Geo": {
        "type": "geo_point"
      }
    }
  }
}
}
'''

es.indices.create(index="crashes", ignore=400, body=mapping)



In [ ]:
crashes = []
i = 0
with open(datapath, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None)  # skip the headers
    for row in reader:
        i += 1
        
        crash = {
            "id": row[0],
            "Geo": row[28]+","+row[29]
        }
        
        crashes.append(crash)
#         if i == 10:
#             break

# print(crashes)
helpers.bulk(es, crashes, index='crashes', doc_type='crash', refresh = True)


In [ ]:
# Bulk Upload Traffic Data

with open(datapath) as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='crashes', doc_type='crash', refresh = True)
  